In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import miepython as mie
from fractions import Fraction
import os
import requests
import re
import sympy as sym
from scipy import integrate, optimize
from copy import deepcopy
from joblib import Parallel, delayed
import multiprocessing

# Read data definition

In [ ]:
def read_optical_data(file_path_or_url, skiprows, convert_lambda=True, custom_names=None):
    # Tải file nếu là URL
    if file_path_or_url.startswith("http://") or file_path_or_url.startswith("https://"):
        local_filename = file_path_or_url.split("/")[-1]
        response = requests.get(file_path_or_url)
        with open(local_filename, 'wb') as f:
            f.write(response.content)
        file_path = local_filename
    else:
        file_path = file_path_or_url

    try:
        with open(file_path, 'r') as f:
            lines = f.readlines()

        # Lấy dòng dữ liệu để xác định số cột
        data_line = lines[skiprows].strip()
        data_items = data_line.split()
        num_columns = len(data_items)

        # Giả sử không có tiêu đề
        use_skiprows = skiprows

        if custom_names:
            if len(custom_names) != num_columns:
                raise ValueError(f"Số lượng tên cột ({len(custom_names)}) không khớp với số cột dữ liệu ({num_columns})")
            column_names = custom_names
        else:
            # Kiểm tra dòng trên skiprows có phải là tiêu đề không
            header_line = lines[skiprows - 1].strip()
            header_items = header_line.split()
            if not header_line.startswith("#") and len(header_items) == num_columns:
                column_names = header_items
                use_skiprows = skiprows
            else:
                column_names = [f"col{i+1}" for i in range(num_columns)]

        # Đọc dữ liệu
        df = pd.read_csv(
            file_path,
            header=None,
            comment="#",
            delim_whitespace=True,
            skiprows=use_skiprows,
            engine='python',
            names=column_names
        )

        # Chuyển đơn vị lambda nếu có
        for name in column_names:
            if name.lower() == "lambda" and convert_lambda:
                df[name] = 0.0001 * df[name]

        return df

    except Exception as e:
        print(f"Đã xảy ra lỗi khi đọc file: {e}")
        return None


# Read parameters from file

In [ ]:
def read_parameters_from_file(filename):
    with open(filename, 'r') as f:
        first_line = f.readline()
        match = re.search(r'Poro=\s*([\d.]+)\s+f_Fe=\s*([\d.]+)\s+b/a\s*=\s*([\d.]+)', first_line)
        if match:
            poro = float(match.group(1))
            f_fe = float(match.group(2))
            b_over_a = float(match.group(3))
            return poro, f_fe, b_over_a
        else:
            return None


# Plot refractive index definition

In [ ]:
def refractive_index(
    df,
    x_col,
    y_cols,
    title="",
    xscale="log",
    yscales=None,
    xlim=None,
    ylims=None,
    xlabel=None,
    ylabels=None,
    colors=None,
    legend_labels=None
):
    n_plots = len(y_cols)
    fig, axes = plt.subplots(n_plots, 1, figsize=(8, 4 * n_plots), sharex=True)

    if n_plots == 1:
        axes = [axes]  # Đảm bảo axes là list nếu chỉ có 1 biểu đồ

    for i, ax in enumerate(axes):
        y_col = y_cols[i]
        yscale = yscales[i] if yscales else "linear"
        ylim = ylims[i] if ylims else None
        ylabel = ylabels[i] if ylabels else y_col
        color = colors[i] if colors else "blue"
        legend = legend_labels[i] if legend_labels else y_col

        # Vẽ đường biểu diễn
        ax.plot(df[x_col]*1e4, df[y_col], color=color, label=legend)

        # Tô nền vùng bước sóng khả kiến (0.38 – 0.76 µm)
        ax.axvspan(0.38, 0.76, color="gray", alpha=0.3)

        ax.set_ylabel(ylabel)
        ax.set_yscale(yscale)
        if ylim:
            ax.set_ylim(ylim)
        ax.grid(True, which="both", ls=":")
        ax.legend()

    axes[-1].set_xlabel(xlabel if xlabel else x_col)
    if xscale:
        axes[-1].set_xscale(xscale)
    if xlim:
        axes[-1].set_xlim(xlim)

    fig.suptitle(title, fontsize=14)
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()


# Mie theory definition

In [ ]:
def mie_theory(df, wavelength, n ,k  , radius):
    # Tạo chiết suất phức
    m_complex = df[n] + 1j * df[k]
    
    Q_abs = []
    Q_ext = []
    Q_sca = []

    for lam, m in zip(df[wavelength], m_complex):
        x = 2 * np.pi * radius / lam  # kích thước không thứ nguyên
        qext, qsca, qback, g = mie.efficiencies_mx(m, x)
        qabs = qext - qsca
        Q_abs.append(qabs)
        Q_ext.append(qext)
        Q_sca.append(qsca)

    Q_abs = np.array(Q_abs)
    Q_ext = np.array(Q_ext)
    Q_sca = np.array(Q_sca)

    return Q_abs, Q_ext, Q_sca


# Maxwell-Garnett Mixing rule definition

In [ ]:
# Apply the Maxwell-Garnett Mixing rule for mass fraction
def mixing_mg(eps_base, eps_incl, mass_incl,rho_h,rho_incl):
    vol_incl = mass_incl * rho_h/rho_incl
    f_c = 1-vol_incl
    numerator = 2 * (1 - f_c) * eps_base + (1 + 2 * f_c) * eps_incl;
    denominator = (2 + f_c) * eps_base + (1 - f_c) * eps_incl;
    e_eff = eps_base * numerator/denominator
    n_eff = np.sqrt(e_eff)
                    
    # Tách phần thực và phần ảo
    real = n_eff.apply(lambda x: x.real)
    imag = n_eff.apply(lambda x: x.imag)
    
    combined = pd.DataFrame({
    'lambda': df_interp["lambda"],
    'n' : real, 
    'k' : imag, })
    
    return combined

# Radius of new core

In [ ]:
def radius_mix(r_c, mass_incl,rho_h,rho_incl):
    vol_incl = mass_incl * rho_h/rho_incl
    d = ((vol_incl * r_c**(3)) + r_c**(3))**(1/3) - r_c 
    return r_c +d

# Planck function for wavelength B(lambda,T)

In [ ]:
def planck(y, time): # erg/s/cm^2/cm/steradian
    exponent = h * c / (y * kB * time)
    safe_exponent = np.clip(exponent, None, 700)
    return (2 * h * c**2 / y**5) * (1. / (np.exp(safe_exponent) - 1.))

# Percentage of mass corresponding volume of mantle/grain

In [ ]:
def per_mass(x,rho_h,rho_incl):
    # x is percentage of volume mantle/grain
    mass = []
    len_x = len(x)
    for i in range (len_x):
        m = (rho_incl/rho_h) / (1/x[i]-1)
        mass.append(m) 
        
    per_mass = pd.DataFrame({
        'per_volume' : x,
        'mass_fraction' : mass,
    })
    return per_mass

## Avaiable data (Thay đổi tham số tại dây)

In [ ]:
# Radius of core
r = np.logspace(np.log10(1e-2),np.log10(1), 200)*1e-4 ##unit of cm 
radius_cm = r #0.1*1e-4 #cm

# Density
rho_m = 0.92 #g/cm^3
rho_s = 3.74 #g/cm^3

# Mass fraction 
mf_m = [0.1,0.2,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.8,0.9,1] # mantle/silicat
mf_m = np.array(mf_m, dtype=float)
# Volume fraction
#vf_m = mf_m*rho_s/rho_m

# Bước sóng cần tìm: 0.000055 cm
lambda_target = 0.55 *1e-4

# Constant
h = 6.626e-27      # Planck constant (erg·s)
c = 3e10        # tốc độ ánh sáng (cm/s)
kB = 1.381e-16      # Boltzmann (erg/K)
T_cmb = 2.725    # Temperture CMB (K)
o_SB = 5.6704e-5   # Stefan boltzmann constant (erg/cm²/s/K⁴)

### Mantle data

In [ ]:
# Read data of mantle
water = read_optical_data("h2o-w-Warren2008.download", skiprows=19, custom_names = ["lambda", "n", "k"] )

### Core data

In [ ]:
# Read data of silicate core

#url = "Astrodust/index_DH21Ad_P0.00_0.00_1.400"


data = read_optical_data("Astrosilicate_Laor_Draine_1993.in", skiprows=6, custom_names = ["lambda", "n_eps", "k_eps", "n", "k"] )
#data = read_optical_data(url, skiprows=2, custom_names = ["lambda", "n", "k", "n_eps", "k_eps"] )

#result = read_parameters_from_file(url)
#Poro = result[0]
#f_Fe = result[1]
#ba = result[2]

data["n"] = data["n"] +1
data["n_eps"] = data["n_eps"] + 1

### Reflective index of mantle

In [ ]:
# Plot refractive index of mantle
refractive_index(
    df=water,
    x_col="lambda",
    y_cols=["n", "k"],
    title=r"Refractive index of the mantle as a complex function",
    xscale="log",
    yscales=["linear", "log"],
    xlim=None,
    ylims=[(0, 3), (1e-12, 1e1)],
    xlabel=r"$\lambda$ [$\mu$m]",
    ylabels=["n", "k"],
    colors=["blue", "blue"],
    legend_labels=["Real", "Imagine"]
)


### Reflective index of core

In [ ]:
# Plot refractive index of silicate core
refractive_index(
    df=data,
    x_col="lambda",
    y_cols=["n", "k"],
    title=r"Refractive index of the silicate core $\mathrm{MgFeSiO_4} [3.74 g/cm^3]$ as a complex function",
    #title=r"Refractive index of the silicate core astrodust Poro= {Poro} f_Fe= {f_Fe} b/a = {ba}",
    xscale="log",
    yscales=["linear", "log"],
    xlim=None,
    ylims=[(0, 6), (1e-8, 10e1)],
    xlabel=r"$\lambda$ [$\mu$m]",
    ylabels=["n", "k"],
    colors=["red", "red"],
    legend_labels=["Real", "Imagine"]
)

### Absorption coefficient over radius of mantle

In [ ]:
from joblib import Parallel, delayed
import multiprocessing

num_cores = multiprocessing.cpu_count()

# Chạy song song cho từng bán kính
results = Parallel(n_jobs=num_cores)(
    delayed(mie_theory)(
        df=water,
        wavelength="lambda",
        n="n",
        k="k",
        radius=radius_cm[i]
    ) for i in range(len(radius_cm))
)

# Tách kết quả thành các mảng Q_abs, Q_ext, Q_sca cho từng bán kính
Q_abs_water_list = [res[0] for res in results]
Q_ext_water_list = [res[1] for res in results]
Q_sca_water_list = [res[2] for res in results]

In [ ]:
# Vẽ đồ thị
plt.figure(figsize=(8, 6))
plt.plot(water["lambda"]*1e4, Q_abs_water_list[1]/radius_cm[1])
plt.xscale("log")
plt.yscale("log")
plt.xlabel(r"$\lambda$ [$\mu$m]")
plt.ylabel(r"$Q_{\mathrm{abs}} / a$ [cm$^{-1}$]")
plt.title(r"Absorption coefficient over radius of mantle")
plt.legend()
plt.grid(True, which="both", ls=":")
plt.axvspan(0.38, 0.76, color="gray", alpha=0.3)
plt.show()

### Absorption coefficient over radius of silicate core

In [ ]:
from joblib import Parallel, delayed
import multiprocessing

num_cores = multiprocessing.cpu_count()

# Chạy song song cho từng bán kính của silicate core
results_sili = Parallel(n_jobs=num_cores)(
    delayed(mie_theory)(
        df=data,
        wavelength="lambda",
        n="n",
        k="k",
        radius=radius_cm[i]
    ) for i in range(len(radius_cm))
)

# Tách kết quả thành các mảng Q_abs, Q_ext, Q_sca cho từng bán kính
Q_abs_sili_list = [res[0] for res in results_sili]
Q_ext_sili_list = [res[1] for res in results_sili]
Q_sca_sili_list = [res[2] for res in results_sili]

In [ ]:
# Vẽ đồ thị
plt.figure(figsize=(8, 6))
plt.plot(data["lambda"]*1e4, Q_abs_sili_list[1]/radius_cm[1], color = "red")
plt.xscale("log")
plt.yscale("log")
plt.xlabel(r"$\lambda$ [$\mu$m]")
plt.ylabel(r"$Q_{\mathrm{abs}} / a$ [cm$^{-1}$]")
plt.title(r"Absorption coefficient over radius of $\mathrm{MgFeSiO_4} [3.74 g/cm^3]$")
plt.grid(True, which="both", ls=":")
plt.legend()
plt.axvspan(0.38, 0.76, color="gray", alpha=0.3)
plt.show()

## Interpo1d

In [ ]:
print(water["lambda"].min(), water["lambda"].max(), data["lambda"].min(), data["lambda"].max())

In [ ]:
# Làm sạch: chuyển lambda về float và đổi về cùng đơn vị nếu cần
data['lambda'] = data['lambda'].astype(float)
water['lambda'] = water['lambda'].astype(float)

# Tạo dải lambda chung bằng cách lấy phần giao hoặc lấy dải mới nếu cần
#lambda_common = np.intersect1d(data['lambda'], water['lambda'])

# Nội suy theo 1 dải:
lambda_common = np.logspace(np.log10(water["lambda"].min()),np.log10(data["lambda"].max()),1000)


# Tạo hàm nội suy
f_n_pyrmg = interp1d(data['lambda'], data['n'], kind='linear', bounds_error=False, fill_value='extrapolate')
f_k_pyrmg = interp1d(data['lambda'], data['k'], kind='linear', bounds_error=False, fill_value='extrapolate')

f_n_h2o = interp1d(water['lambda'], water['n'], kind='linear', bounds_error=False, fill_value='extrapolate')
f_k_h2o = interp1d(water['lambda'], water['k'], kind='linear', bounds_error=False, fill_value='extrapolate')

# Tạo bảng mới với các giá trị nội suy
df_interp = pd.DataFrame({
    'lambda': lambda_common,
    'n_pyrmg': f_n_pyrmg(lambda_common),
    'k_pyrmg': f_k_pyrmg(lambda_common),
    'n_h2o': f_n_h2o(lambda_common),
    'k_h2o': f_k_h2o(lambda_common)
})

In [ ]:
plt.figure()
plt.plot(data["lambda"], data["k"], color='blue', label='k_core')
plt.plot(df_interp["lambda"], df_interp["k_pyrmg"], color='red', label='k_mix', ls= "--")
plt.plot(water["lambda"], water["k"], color='green', label='k_water')
plt.plot(df_interp["lambda"], df_interp["k_h2o"], color='red', ls= "--")
plt.xscale("log")
plt.yscale("log")
plt.xlabel(r"$\lambda$ [$\mu$m]")
plt.ylabel(r"k")
plt.xlim(1e-6,1)
plt.ylim(1e-3,2)
plt.title(r"Range of common wavelength")
plt.legend()
plt.grid(True, which="both", ls=":")
plt.show()

## Mixing 

In [ ]:
# Reflection of silicat and ice
m1 = df_interp['n_pyrmg'] + 1j*df_interp['k_pyrmg']
m2 = df_interp['n_h2o'] + 1j*df_interp['k_h2o']

# Dielectric constant of silicat and ice
e1 = m1**2
e2 = m2**2

# Tạo dictionary chứa các bảng theo tên
combined_dict = {}

for i, mf in enumerate(mf_m):
    name = f"df_combined_{i+1}"
    combined_dict[name] = mixing_mg(e2, e1, mf, rho_s, rho_m)

# Ví dụ: Truy cập từng bảng
# ...existing code...
df_combined = combined_dict["df_combined_1"]
df_combined_2 = combined_dict["df_combined_2"]
df_combined_3 = combined_dict["df_combined_3"]
df_combined_4 = combined_dict["df_combined_4"]
df_combined_5 = combined_dict["df_combined_5"]
df_combined_6 = combined_dict["df_combined_6"]
df_combined_7 = combined_dict["df_combined_7"]
df_combined_8 = combined_dict["df_combined_8"]
df_combined_9 = combined_dict["df_combined_9"]
df_combined_10 = combined_dict["df_combined_10"]
df_combined_11 = combined_dict["df_combined_11"]
df_combined_12 = combined_dict["df_combined_12"]
df_combined_13 = combined_dict["df_combined_13"]
df_combined_14 = combined_dict["df_combined_14"]
# Xóa hoặc comment các dòng dưới nếu mf_m chỉ có 14 phần tử
# df_combined_15 = combined_dict["df_combined_15"]
# ...

In [ ]:
# Plot refractive index of silicate core
refractive_index(
    df=df_combined,
    x_col="lambda",
    y_cols=["n", "k"],
    title=f"Refractive index of combined as a complex functio, mass fraction = {mf_m[0]}",
    xscale="log",
    yscales=["linear", "log"],
    xlim=None,
    ylims=[(0.5, 2.5), (1e-3, 1.5)],
    xlabel=r"$\lambda$ [$\mu$m]",
    ylabels=["n", "k"],
    colors=["black", "black"],
    legend_labels=["Real", "Imagine"]
)

In [ ]:

plt.figure()
plt.plot(df_interp["lambda"]*1e4, df_interp["n_h2o"], color='blue', label='n_mantle',ls="--")
plt.plot(df_interp["lambda"]*1e4, df_interp["n_pyrmg"], color='red', label='n_silicat', ls= "--")
plt.plot(df_combined["lambda"]*1e4, df_combined["n"], color='black', label='n_combine',ls="-" )
plt.xscale("log")
plt.xlim(1e-1,1e3)
plt.xlabel(r"$\lambda$ [$\mu$m]")
plt.ylabel(r"n")
plt.title(r"Compare n_mantle, n_silicat and n_combined")
plt.legend()
plt.grid(True, which="both", ls=":")
# Tô vùng khả kiến từ 0.38 đến 0.76 µm
plt.axvspan(0.38, 0.76, color="gray", alpha=0.3)

plt.show()

In [ ]:
plt.figure()
plt.plot(df_interp["lambda"]*1e4, df_interp["k_h2o"], color='blue', label='k_mantle',ls="--")
plt.plot(df_interp["lambda"]*1e4, df_interp["k_pyrmg"], color='red', label='k_silicat', ls= "--")
plt.plot(df_combined["lambda"]*1e4, df_combined["k"], color='black', label='k_combine',ls="-" )
plt.xscale("log")
plt.yscale("log")
plt.xlabel(r"$\lambda$ [$\mu$m]")
plt.ylabel(r"k")
plt.ylim(1e-3,1e1)
plt.title(r"Compare k_mantle, k_silicat and k_combined")
plt.legend()
plt.grid(True, which="both", ls=":")
# Tô vùng khả kiến từ 0.38 đến 0.76 µm
plt.axvspan(0.38, 0.76, color="gray", alpha=0.3)

plt.show()

### Find radius mix

In [ ]:
# Find radius mix
#radius_mix_cm = radius_mix(radius_cm,mf_m,rho_s,rho_m)



# Tạo dictionary chứa các bảng theo tên
combined_radi = {}

for i, mf in enumerate(mf_m):
    name = f"radius_mix_cm_{i+1}"
    combined_radi[name] = radius_mix(radius_cm,mf,rho_s,rho_m)

# Ví dụ: Truy cập từng bảng
radius_mix_cm = combined_radi["radius_mix_cm_1"]
radius_mix_cm_2 = combined_radi["radius_mix_cm_2"]
radius_mix_cm_3 = combined_radi["radius_mix_cm_3"]
radius_mix_cm_4 = combined_radi["radius_mix_cm_4"]
radius_mix_cm_5 = combined_radi["radius_mix_cm_5"]
radius_mix_cm_6 = combined_radi["radius_mix_cm_6"]
radius_mix_cm_7 = combined_radi["radius_mix_cm_7"]
radius_mix_cm_8 = combined_radi["radius_mix_cm_8"]
radius_mix_cm_9 = combined_radi["radius_mix_cm_9"]
radius_mix_cm_10 = combined_radi["radius_mix_cm_10"]
radius_mix_cm_11 = combined_radi["radius_mix_cm_11"]
radius_mix_cm_12 = combined_radi["radius_mix_cm_12"]
radius_mix_cm_13 = combined_radi["radius_mix_cm_13"]
radius_mix_cm_14 = combined_radi["radius_mix_cm_14"]
len_mix = len(radius_mix_cm)

In [ ]:
from joblib import Parallel, delayed
import multiprocessing

num_cores = multiprocessing.cpu_count()

# Song song cho từng trường hợp
results_mix_1 = Parallel(n_jobs=num_cores)(
    delayed(mie_theory)(
        df=df_combined,
        wavelength="lambda",
        n="n",
        k="k",
        radius=radius_mix_cm[i]
    ) for i in range(len(radius_mix_cm))
)
Q_abs_mix_sili_list = [res[0] for res in results_mix_1]
Q_ext_mix_sili_list = [res[1] for res in results_mix_1]
Q_sca_mix_sili_list = [res[2] for res in results_mix_1]

results_mix_2 = Parallel(n_jobs=num_cores)(
    delayed(mie_theory)(
        df=df_combined_2,
        wavelength="lambda",
        n="n",
        k="k",
        radius=radius_mix_cm_2[i]
    ) for i in range(len(radius_mix_cm_2))
)
Q_abs_mix_sili_2_list = [res[0] for res in results_mix_2]
Q_ext_mix_sili_2_list = [res[1] for res in results_mix_2]
Q_sca_mix_sili_2_list = [res[2] for res in results_mix_2]

results_mix_3 = Parallel(n_jobs=num_cores)(
    delayed(mie_theory)(
        df=df_combined_3,
        wavelength="lambda",
        n="n",
        k="k",
        radius=radius_mix_cm_3[i]
    ) for i in range(len(radius_mix_cm_3))
)
Q_abs_mix_sili_3_list = [res[0] for res in results_mix_3]
Q_ext_mix_sili_3_list = [res[1] for res in results_mix_3]
Q_sca_mix_sili_3_list = [res[2] for res in results_mix_3]

results_mix_4 = Parallel(n_jobs=num_cores)(
    delayed(mie_theory)(
        df=df_combined_4,
        wavelength="lambda",
        n="n",
        k="k",
        radius=radius_mix_cm_4[i]
    ) for i in range(len(radius_mix_cm_4))
)
Q_abs_mix_sili_4_list = [res[0] for res in results_mix_4]
Q_ext_mix_sili_4_list = [res[1] for res in results_mix_4]
Q_sca_mix_sili_4_list = [res[2] for res in results_mix_4]

results_mix_5 = Parallel(n_jobs=num_cores)(
    delayed(mie_theory)(
        df=df_combined_5,
        wavelength="lambda",
        n="n",
        k="k",
        radius=radius_mix_cm_5[i]
    ) for i in range(len(radius_mix_cm_5))
)
Q_abs_mix_sili_5_list = [res[0] for res in results_mix_5]
Q_ext_mix_sili_5_list = [res[1] for res in results_mix_5]
Q_sca_mix_sili_5_list = [res[2] for res in results_mix_5]

results_mix_6 = Parallel(n_jobs=num_cores)(
    delayed(mie_theory)(
        df=df_combined_6,
        wavelength="lambda",
        n="n",
        k="k",
        radius=radius_mix_cm_6[i]
    ) for i in range(len(radius_mix_cm_6))
)
Q_abs_mix_sili_6_list = [res[0] for res in results_mix_6]
Q_ext_mix_sili_6_list = [res[1] for res in results_mix_6]
Q_sca_mix_sili_6_list = [res[2] for res in results_mix_6]

results_mix_7 = Parallel(n_jobs=num_cores)(
    delayed(mie_theory)(
        df=df_combined_7,
        wavelength="lambda",
        n="n",
        k="k",
        radius=radius_mix_cm_7[i]
    ) for i in range(len(radius_mix_cm_7))
)
Q_abs_mix_sili_7_list = [res[0] for res in results_mix_7]
Q_ext_mix_sili_7_list = [res[1] for res in results_mix_7]
Q_sca_mix_sili_7_list = [res[2] for res in results_mix_7]

results_mix_8 = Parallel(n_jobs=num_cores)(
    delayed(mie_theory)(
        df=df_combined_8,
        wavelength="lambda",
        n="n",
        k="k",
        radius=radius_mix_cm_8[i]
    ) for i in range(len(radius_mix_cm_8))
)
Q_abs_mix_sili_8_list = [res[0] for res in results_mix_8]
Q_ext_mix_sili_8_list = [res[1] for res in results_mix_8]
Q_sca_mix_sili_8_list = [res[2] for res in results_mix_8]
results_mix_9 = Parallel(n_jobs=num_cores)(
    delayed(mie_theory)(
        df=df_combined_9,
        wavelength="lambda",
        n="n",
        k="k",
        radius=radius_mix_cm_9[i]
    ) for i in range(len(radius_mix_cm_9))
)
Q_abs_mix_sili_9_list = [res[0] for res in results_mix_9]
Q_ext_mix_sili_9_list = [res[1] for res in results_mix_9]
Q_sca_mix_sili_9_list = [res[2] for res in results_mix_9]
results_mix_10 = Parallel(n_jobs=num_cores)(
    delayed(mie_theory)(
        df=df_combined_10,
        wavelength="lambda",
        n="n",
        k="k",
        radius=radius_mix_cm_10[i]
    ) for i in range(len(radius_mix_cm_10))
)
Q_abs_mix_sili_10_list = [res[0] for res in results_mix_10]
Q_ext_mix_sili_10_list = [res[1] for res in results_mix_10]
Q_sca_mix_sili_10_list = [res[2] for res in results_mix_10]
results_mix_11 = Parallel(n_jobs=num_cores)(
    delayed(mie_theory)(
        df=df_combined_11,
        wavelength="lambda",
        n="n",
        k="k",
        radius=radius_mix_cm_11[i]
    ) for i in range(len(radius_mix_cm_11))
)
Q_abs_mix_sili_11_list = [res[0] for res in results_mix_11]
Q_ext_mix_sili_11_list = [res[1] for res in results_mix_11]
Q_sca_mix_sili_11_list = [res[2] for res in results_mix_11]
results_mix_12 = Parallel(n_jobs=num_cores)(
    delayed(mie_theory)(
        df=df_combined_12,
        wavelength="lambda",
        n="n",
        k="k",
        radius=radius_mix_cm_12[i]
    ) for i in range(len(radius_mix_cm_12))
)
Q_abs_mix_sili_12_list = [res[0] for res in results_mix_12]
Q_ext_mix_sili_12_list = [res[1] for res in results_mix_12]
Q_sca_mix_sili_12_list = [res[2] for res in results_mix_12]
results_mix_13 = Parallel(n_jobs=num_cores)(
    delayed(mie_theory)(
        df=df_combined_13,
        wavelength="lambda",
        n="n",
        k="k",
        radius=radius_mix_cm_13[i]
    ) for i in range(len(radius_mix_cm_13))
)
Q_abs_mix_sili_13_list = [res[0] for res in results_mix_13]
Q_ext_mix_sili_13_list = [res[1] for res in results_mix_13]
Q_sca_mix_sili_13_list = [res[2] for res in results_mix_13]
results_mix_14 = Parallel(n_jobs=num_cores)(
    delayed(mie_theory)(
        df=df_combined_14,
        wavelength="lambda",
        n="n",
        k="k",
        radius=radius_mix_cm_14[i]
    ) for i in range(len(radius_mix_cm_14))
)
Q_abs_mix_sili_14_list = [res[0] for res in results_mix_14]
Q_ext_mix_sili_14_list = [res[1] for res in results_mix_14]
Q_sca_mix_sili_14_list = [res[2] for res in results_mix_14]


In [ ]:
# Vẽ đồ thị cho từng trường hợp
plt.figure(figsize=(10, 6))
plt.plot(df_interp["lambda"]*1e4, Q_abs_mix_sili_list[1]/radius_mix_cm[1], color='black', label='Q_abs_mixture_1')
plt.plot(df_interp["lambda"]*1e4, Q_abs_mix_sili_2_list[1]/radius_mix_cm_2[1], color='blue', label='Q_abs_mixture_2')
plt.plot(df_interp["lambda"]*1e4, Q_abs_mix_sili_3_list[1]/radius_mix_cm_3[1], color='red', label='Q_abs_mixture_3')
plt.plot(df_interp["lambda"]*1e4, Q_abs_mix_sili_4_list[1]/radius_mix_cm_4[1], color='green', label='Q_abs_mixture_4')
plt.plot(df_interp["lambda"]*1e4, Q_abs_mix_sili_5_list[1]/radius_mix_cm_5[1], color='orange', label='Q_abs_mixture_5')
plt.plot(df_interp["lambda"]*1e4, Q_abs_mix_sili_6_list[1]/radius_mix_cm_6[1], color='purple', label='Q_abs_mixture_6')
plt.plot(df_interp["lambda"]*1e4, Q_abs_mix_sili_7_list[1]/radius_mix_cm_7[1], color='brown', label='Q_abs_mixture_7')
plt.plot(df_interp["lambda"]*1e4, Q_abs_mix_sili_8_list[1]/radius_mix_cm_8[1], color='pink', label='Q_abs_mixture_8')
plt.plot(df_interp["lambda"]*1e4, Q_abs_mix_sili_9_list[1]/radius_mix_cm_9[1], color='gray', label='Q_abs_mixture_9')
plt.plot(df_interp["lambda"]*1e4, Q_abs_mix_sili_10_list[1]/radius_mix_cm_10[1], color='cyan', label='Q_abs_mixture_10')
plt.plot(df_interp["lambda"]*1e4, Q_abs_mix_sili_11_list[1]/radius_mix_cm_11[1], color='magenta', label='Q_abs_mixture_11')
plt.plot(df_interp["lambda"]*1e4, Q_abs_mix_sili_12_list[1]/radius_mix_cm_12[1], color='olive', label='Q_abs_mixture_12')
plt.plot(df_interp["lambda"]*1e4, Q_abs_mix_sili_13_list[1]/radius_mix_cm_13[1], color='teal', label='Q_abs_mixture_13')
plt.plot(df_interp["lambda"]*1e4, Q_abs_mix_sili_14_list[1]/radius_mix_cm_14[1], color='navy', label='Q_abs_mixture_14')
plt.xscale("log")
plt.yscale("log")
plt.xlabel(r"$\lambda$ [$\mu$m]")
plt.ylabel(r"$Q_{\mathrm{abs}} / a$ [cm$^{-1}$]")
plt.xlim(1e-2,1e4)
plt.ylim(1e-2,1e7)
plt.title(r"Absorption coefficient over radius of silicate core with different mass fraction")
plt.legend()
plt.grid(True, which="both", ls=":")
plt.axvspan(0.38, 0.76, color="gray", alpha=0.3)
plt.show()
import numpy as np
def refractive_index(df, x_col, y_cols, title, xscale=None, yscales=None, xlim=None, ylims=None, xlabel=None, ylabels=None, colors=None, legend_labels=None):
    import matplotlib.pyplot as plt

    fig, axes = plt.subplots(nrows=len(y_cols), ncols=1, figsize=(10, 6), sharex=True)

    for i, y_col in enumerate(y_cols):
        axes[i].plot(df[x_col], df[y_col], label=legend_labels[i] if legend_labels else y_col, color=colors[i] if colors else None)
        axes[i].set_ylabel(ylabels[i] if ylabels else y_col)
        axes[i].set_yscale(yscales[i] if yscales else 'linear')
        if ylims:
            axes[i].set_ylim(ylims[i])
        axes[i].grid(True)

    if len(y_cols) > 1:
        axes[-1].legend()
    axes[-1].set_xlabel(xlabel if xlabel else x_col)
    if xscale:
        axes[-1].set_xscale(xscale)
    if xlim:
        axes[-1].set_xlim(xlim)
    fig.suptitle(title)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
def read_optical_data(file_path, skiprows=0, custom_names=None):
    import pandas as pd

    # Đọc dữ liệu từ file
    df = pd.read_csv(file_path, skiprows=skiprows, delim_whitespace=True, header=None)

    # Nếu có custom_names, đặt tên cột
    if custom_names:
        df.columns = custom_names
    else:
        df.columns = ['lambda', 'n', 'k']

    # Chuyển đổi cột 'lambda' sang float
    df['lambda'] = df['lambda'].astype(float)

    return df
import pandas as pd
import numpy as np
def mixing_mg(e2, e1, mass_fraction, rho_s, rho_m):
    """
    Calculate the effective refractive index for a mixture of silicate and water.
    
    Parameters:
    - e2: Dielectric constant of silicate core
    - e1: Dielectric constant of water
    - mass_fraction: Mass fraction of silicate in the mixture
    - rho_s: Density of silicate core
    - rho_m: Density of water
    
    Returns:
    - DataFrame with wavelength, real part (n), and imaginary part (k) of the effective refractive index.
    """
    
    # Calculate the effective dielectric constant
    e_eff = (mass_fraction * e2 + (1 - mass_fraction) * e1) / (mass_fraction * rho_s + (1 - mass_fraction) * rho_m)
    
    # Calculate the effective refractive index
    n_eff = np.sqrt(e_eff)
    
    # Extract real and imaginary parts
    # Assuming e_eff is complex, we can separate real and imaginary parts
    # If e_eff is real, then k_eff will be zero
    # k_eff = np.zeros_like(n_eff)  # Assuming no absorption for simplicity
    # k_eff = np.imag(np.sqrt(e_eff))  # Extract imaginary part if e_eff is complex
    # k_eff = np.zeros_like(n_eff)  # Assuming no absorption for simplicity

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(df_combined["lambda"]*1e4, Q_abs_mix_sili_list[1]/(radius_mix_cm[1]*1e4), label = fr"Mixing, f_m = {mf_m[0]}, a= {radius_mix_cm[1]*1e4:.3f} $\mu$m")
plt.plot(df_combined["lambda"]*1e4, Q_abs_mix_sili_list[60]/(radius_mix_cm[60]*1e4), label = fr"Mixing, f_m = {mf_m[0]}, a= {radius_mix_cm[60]*1e4:.3f} $\mu$m")
plt.plot(df_combined["lambda"]*1e4, Q_abs_mix_sili_list[120]/(radius_mix_cm[120]*1e4), label = fr"Mixing, f_m = {mf_m[0]}, a= {radius_mix_cm[120]*1e4:.3f} $\mu$m")
plt.plot(df_combined["lambda"]*1e4, Q_abs_mix_sili_list[199]/(radius_mix_cm[199]*1e4), label = fr"Mixing, f_m = {mf_m[0]}, a= {radius_mix_cm[199]*1e4:.3f} $\mu$m")
plt.xscale("log")
plt.yscale("log")
plt.xlabel(r"$\lambda$ [$\mu$m]")
plt.ylabel(r"$Q_{\mathrm{abs}} / a$ [$\mu$m$^{-1}$]")
plt.xlim(1e-1, 1e3)
plt.ylim(1e-4, 500)
plt.title(r"Absorption coefficient over radius")
plt.legend()
plt.grid(True, which="both", ls=":")
plt.axvspan(0.38, 0.76, color="gray", alpha=0.3)
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
#plt.plot(water["lambda"]*1e4, Q_abs_water/(radius_cm*1e4), color='blue', label =f"mantle (d = {(radius_mix_cm - radius_cm)*1e4:.3f} $\mu$m)", ls ="--" )
plt.plot(data["lambda"]*1e4, Q_ext_sili_list[1]/(radius_cm[1]*1e4), label = fr"Silicat core (radius = {radius_cm[1]*1e4:.3f} $\mu$m)", ls ="--" )
plt.plot(df_combined["lambda"]*1e4, Q_ext_mix_sili_list[1]/(radius_mix_cm[1]*1e4), color='red', label = fr"Mixing with mf_m = {mf_m[0]:.2f} , radius = {radius_mix_cm[1]*1e4:.3f} $\mu$m")
plt.xscale("log")
plt.yscale("log")
plt.xlabel(r"$\lambda$ [$\mu$m]")
plt.ylabel(r"$Q_{\mathrm{ext}} / a$ [$\mu$m$^{-1}$]")
plt.xlim(0.03,2000)
plt.title(r"Extintion coefficient over radius")
plt.legend()
plt.grid(True, which="both", ls=":")
plt.axvspan(0.38, 0.76, color="gray", alpha=0.3)
plt.show()

# Save data
with open("Absorption coefficient over radius of mix.in", "w") as f:
    for lam, q in zip(df_combined["lambda"]*1e4, Q_abs_mix_sili):
        f.write(f"{lam:.6e}\t{q:.6e}\n")

#Tiêu đề
with open("Absorption coefficient over radius of mix.in", "w") as f:
    f.write("lambda(um)\tQ_abs_mix_sili\n")
    for lam, q in zip(df_combined["lambda"]*1e4, Q_abs_mix_sili):
        f.write(f"{lam:.6e}\t{q:.6e}\n")

## Optical deep


In [ ]:
def optical_deep(df_combined, radius_mix_cm, lambda_target, n, k):
    """
    Tính optical deep tại bước sóng lambda_target
    và trên toàn phổ từ bảng df_combined.

    Tham số:
        df_combined: DataFrame chứa 'lambda', 'n', 'k'
        radius_mix_cm: danh sách bán kính hạt (đơn vị cm)
        lambda_target: bước sóng mục tiêu (cm), mặc định là 0.55 μm = 5.5e-5 cm

    Trả về:
        - tau_v: optical depth tại lambda_target
        - tau_lambda: mảng optical depth theo từng bước sóng
    """
    
    # --- 1. Interpolation tại lambda_target ---
    interp_n = interp1d(df_combined['lambda'], df_combined[n], kind='linear')
    interp_k = interp1d(df_combined['lambda'], df_combined[k], kind='linear')

    n_target = interp_n(lambda_target)
    k_target = interp_k(lambda_target)
    m_complex_target = n_target + 1j * k_target

    Q_ext_target = []
    for r in radius_mix_cm:
        x = 2 * np.pi * r / lambda_target
        qext, qsca, qback, g = mie.efficiencies_mx(m_complex_target, x)
        qabs = qext - qsca
        Q_ext_target.append(qext * r ** (-1.5))

    Q_ext_target = np.array(Q_ext_target)
    tau_v = integrate.trapezoid(Q_ext_target, radius_mix_cm)

    # --- 2. Tính theo toàn phổ bước sóng ---
    tau_lambda = []

    for i in range(len(df_combined['lambda'])):
        lam = df_combined['lambda'][i]
        m = df_combined[n][i] + 1j * df_combined[k][i]

        Q_ext_each = []
        for r in radius_mix_cm:
            x = 2 * np.pi * r / lam
            qext, qsca, qback, g = mie.efficiencies_mx(m, x)
            qabs = qext - qsca
            Q_ext_each.append(qext * r ** (-1.5))

        Q_ext_each = np.array(Q_ext_each)

        if len(Q_ext_each) == len(radius_mix_cm):
            tau = integrate.trapezoid(Q_ext_each, radius_mix_cm)
        else:
            print(f"[!] Lỗi chiều tại lambda = {lam}")
            tau = np.nan

        tau_lambda.append(tau)

    tau_lambda = np.array(tau_lambda)

    return tau_v, tau_lambda


#### For mixing

In [ ]:
from joblib import Parallel, delayed
import multiprocessing

lambda_target = 0.55 * 1e-4

# Danh sách các tham số cho từng trường hợp
params = [
    (df_combined, radius_mix_cm),
    (df_combined_2, radius_mix_cm_2),
    (df_combined_3, radius_mix_cm_3),
    (df_combined_4, radius_mix_cm_4),
    (df_combined_5, radius_mix_cm_5),
    (df_combined_6, radius_mix_cm_6),
    (df_combined_7, radius_mix_cm_7),
    (df_combined_8, radius_mix_cm_8),
    (df_combined_9, radius_mix_cm_9),
    (df_combined_10, radius_mix_cm_10),
    (df_combined_11, radius_mix_cm_11),
    (df_combined_12, radius_mix_cm_12),
    (df_combined_13, radius_mix_cm_13),
    (df_combined_14, radius_mix_cm_14),]

num_cores = multiprocessing.cpu_count()

results = Parallel(n_jobs=num_cores)(
    delayed(optical_deep)(df, r_cm, lambda_target, n='n', k='k')
    for df, r_cm in params
)

# Giải nén kết quả
(tau_v, tau_lambda), \
(tau_v_2, tau_lambda_2), \
(tau_v_3, tau_lambda_3), \
(tau_v_4, tau_lambda_4), \
(tau_v_5, tau_lambda_5), \
(tau_v_6, tau_lambda_6), \
(tau_v_7, tau_lambda_7), \
(tau_v_8, tau_lambda_8), \
(tau_v_9, tau_lambda_9), \
(tau_v_10, tau_lambda_10), \
(tau_v_11, tau_lambda_11), \
(tau_v_12, tau_lambda_12), \
(tau_v_13, tau_lambda_13), \
(tau_v_14, tau_lambda_14) = results

#### For silicat core

In [ ]:
from joblib import Parallel, delayed
import multiprocessing

def optical_deep_parallel(df_combined, radius_mix_cm, lambda_target, n, k):
    # --- 1. Interpolation tại lambda_target ---
    interp_n = interp1d(df_combined['lambda'], df_combined[n], kind='linear')
    interp_k = interp1d(df_combined['lambda'], df_combined[k], kind='linear')

    n_target = interp_n(lambda_target)
    k_target = interp_k(lambda_target)
    m_complex_target = n_target + 1j * k_target

    Q_ext_target = []
    for r in radius_mix_cm:
        x = 2 * np.pi * r / lambda_target
        qext, qsca, qback, g = mie.efficiencies_mx(m_complex_target, x)
        qabs = qext - qsca
        Q_ext_target.append(qext * r ** (-1.5))

    Q_ext_target = np.array(Q_ext_target)
    tau_v = integrate.trapezoid(Q_ext_target, radius_mix_cm)

    # --- 2. Song song hóa tính tau_lambda ---
    def calc_tau(i):
        lam = df_combined['lambda'][i]
        m = df_combined[n][i] + 1j * df_combined[k][i]
        Q_ext_each = []
        for r in radius_mix_cm:
            x = 2 * np.pi * r / lam
            qext, qsca, qback, g = mie.efficiencies_mx(m, x)
            qabs = qext - qsca
            Q_ext_each.append(qext * r ** (-1.5))
        Q_ext_each = np.array(Q_ext_each)
        if len(Q_ext_each) == len(radius_mix_cm):
            tau = integrate.trapezoid(Q_ext_each, radius_mix_cm)
        else:
            tau = np.nan
        return tau

    num_cores = multiprocessing.cpu_count()
    tau_lambda = Parallel(n_jobs=num_cores)(
        delayed(calc_tau)(i) for i in range(len(df_combined['lambda']))
    )
    tau_lambda = np.array(tau_lambda)

    return tau_v, tau_lambda

# Gọi hàm song song:
tau_v_sili, tau_lambda_sili = optical_deep_parallel(df_interp, radius_cm, lambda_target, n='n_pyrmg', k='k_pyrmg')

In [ ]:
# Vẽ đồ thị
plt.figure(figsize=(8, 6))
plt.plot(df_interp["lambda"]*1e4, tau_lambda_sili /tau_v_sili, label = "Silicat core", color = "black", ls = "--") 
plt.plot(df_combined["lambda"]*1e4, tau_lambda /tau_v, label = f"Mixing, f_m = {mf_m[0]}") # label =f"Mixing with mf_m = {mf_m:.2f} , radius = {radius_mix_cm*1e4:.3f} $\mu$m"
#plt.plot(df_combined_2["lambda"]*1e4, tau_lambda_2 /tau_v_2, label = f"Mixing, f_m = {mf_m[1]}")
#plt.plot(df_combined_3["lambda"]*1e4, tau_lambda_3 /tau_v_3, label = f"Mixing, f_m = {mf_m[2]}")
#plt.plot(df_combined_4["lambda"]*1e4, tau_lambda_4 /tau_v_4, label = f"Mixing, f_m = {mf_m[3]}")
#plt.plot(df_combined_5["lambda"]*1e4, tau_lambda_5 /tau_v_5, label = f"Mixing, f_m = {mf_m[4]}")
#plt.plot(df_combined_6["lambda"]*1e4, tau_lambda_6 /tau_v_6, label = f"Mixing, f_m = {mf_m[5]}")
#plt.plot(df_combined_7["lambda"]*1e4, tau_lambda_7 /tau_v_7, label = f"Mixing, f_m = {mf_m[6]}")
plt.xscale("log")
plt.yscale("log")
plt.xlabel(r"$\lambda$ [$\mu$m]")
plt.ylabel(r"$A_{\mathrm{\lambda}}$ / $A_{\mathrm{v}}$")
plt.xlim(0.05,1e3)
plt.ylim(1e-3,20)
plt.title(r"Extinction curve of mixing and silicate core r ") #and mix $\mathrm{MgFeSiO_4} [3.74 g/cm^3]$
plt.legend()
plt.grid(True, which="both", ls=":")
# Tô vùng khả kiến từ 0.38 đến 0.76 µm
plt.axvspan(0.38, 0.76, color="gray", alpha=0.3)

plt.show()

## Specific energy density u_lambda

In [ ]:
def specific_energy_density(df_combined, T_i, W_i,U):
    """
    Tính mật độ năng lượng phổ u_lambda tại mỗi bước sóng.
    
    Tham số:
        - df_combined: DataFrame chứa ít nhất cột 'lambda' (đơn vị cm)
        - T_i: danh sách các nhiệt độ (K) cho các nguồn nền
        - W_i: ma trận hệ số trọng số (n nguồn x n bước sóng)
        - T_cmb: nhiệt độ nền vũ trụ (K)
        - factor: hệ số nhân
    
    Trả về:
        - u_lambda: mảng NumPy chứa mật độ năng lượng phổ tại mỗi bước sóng
    """
    
    wavelengths = df_combined["lambda"].values  # đơn vị cm
    n_lambda = len(wavelengths)

    # Tổng Planck cho nhiều nguồn T_i
    sum_blackbody = []
    for j in range(len(df_combined["lambda"])):  
        total = 0.0
        for i in range(3):  # duyệt qua từng nhiệt độ
            B = np.array([planck(wavelengths, Ti) for Ti in T_i])  # shape (n_T, n_wl)
            total += W_i[i] * B[i][j]  # trọng số * bức xạ tại λ_j, T_i
        sum_blackbody.append(total)
    # Chuyển thành mảng numpy để nhân hằng số
    sum_blackbody = np.array(sum_blackbody, dtype=float)
    # Planck function unit: erg cm⁻² s⁻¹ sr⁻¹ cm⁻¹
    
    # Thành phần UV 
    u_UV = []
    for lam_cm in wavelengths:
        if 1340e-8 < lam_cm < 2460e-8:
            u = 2.373e-14 * (lam_cm * 1e4) ** (-0.6678) # erg·cm⁻³
        elif 1100e-8 < lam_cm <= 1340e-8:
            u = 6.825e-13 * lam_cm * 1e4
        elif 912e-8 < lam_cm <= 1100e-8:
            u = 1.287e-9 * (lam_cm * 1e4) ** (4.4172)
        else:
            u = 0
        u_UV.append(u)
    u_UV = np.array(u_UV, dtype=float)

    # Thành phần Planck từ T_cmb
    planck_Tcmb = planck(wavelengths, T_cmb)

    # Specific energy density
    factor = 4 * np.pi / c #s/cm
    u_lambda = U*(u_UV/wavelengths + factor * sum_blackbody ) + factor * planck_Tcmb 
    u_lambda = np.array(u_lambda, dtype=float)
    return u_lambda

#### Thay đổi các tham số Wi, Ti, U tại đây

In [ ]:
# Avaiable data
W_i = np.array([1e-14, 1.65e-13, 4e-13])  # Weighting factors (dimensionless)
T_i = np.array([7500, 4000, 3000])     # Blackbody temperatures (Kelvin)
U = [1,10,20,50,100,1000]

u_lambda = specific_energy_density(df_combined, T_i, W_i, U[0])
u_lambda_10 = specific_energy_density(df_combined, T_i, W_i, U[1])
u_lambda_20 = specific_energy_density(df_combined, T_i, W_i, U[2])
u_lambda_50 = specific_energy_density(df_combined, T_i, W_i, U[3])
u_lambda_100 = specific_energy_density(df_combined, T_i, W_i, U[4])
u_lambda_1000 = specific_energy_density(df_combined, T_i, W_i, U[5])


In [ ]:
plt.figure()
plt.plot(df_combined["lambda"]*1e4, u_lambda*df_combined["lambda"], label = f'U = {U[0]}')
plt.plot(df_combined["lambda"]*1e4, u_lambda_10*df_combined["lambda"], label = f'U = {U[1]}')
plt.plot(df_combined["lambda"]*1e4, u_lambda_20*df_combined["lambda"], label = f'U = {U[2]}')
plt.plot(df_combined["lambda"]*1e4, u_lambda_50*df_combined["lambda"], label = f'U = {U[3]}')
plt.plot(df_combined["lambda"]*1e4, u_lambda_100*df_combined["lambda"], label = f'U = {U[4]}')
plt.plot(df_combined["lambda"]*1e4, u_lambda_1000*df_combined["lambda"], label = f'U = {U[5]}')
plt.xscale("log")
plt.yscale("log")
plt.xlabel(r"$\lambda$ [$\mu$m]")
plt.ylabel(r"$\lambda u_\lambda$ [erg/$cm^3$]")
plt.xlim(9.12e-2,20)
plt.ylim(1e-16,1e-9)
plt.title(r"Specific energy density $u_\lambda$")
plt.legend()
plt.grid(True, which="both", ls=":")
# Tô vùng khả kiến từ 0.38 đến 0.76 µm
plt.axvspan(0.38, 0.76, color="gray", alpha=0.3)

plt.show()

In [ ]:
# Chọn vùng bước sóng từ 13.6eV đến 20 micron
lambda_ext = np.logspace(np.log10(9.12e-6),np.log10(2e-3),200)

# Tạo hàm nội suy
u_lambda_ext = interp1d( df_combined['lambda'], u_lambda, kind='linear', bounds_error=False, fill_value='extrapolate')

# Tạo bảng mới với các giá trị nội suy
data_ext = pd.DataFrame({
    'lambda': lambda_ext,
    'u_lambda' : u_lambda_ext(lambda_ext), 
})

In [ ]:
plt.figure()
plt.plot(df_combined["lambda"]*1e4, u_lambda, color='blue', label = r"$u_\lambda$ intinal")
plt.plot(data_ext["lambda"]*1e4, data_ext["u_lambda"], color = "red", ls='--', label = r"$u_\lambda$ cut")
plt.xscale("log")
plt.yscale("log")
plt.xlabel(r"$\lambda$ [$\mu$m]")
plt.ylabel(r"$\lambda u_\lambda$ [erg/$cm^3$]")
plt.title(r"Specific energy density $u_\lambda$")
plt.legend()
plt.xlim(9.12e-2,20)
plt.ylim(1e-13,1e-7)
plt.grid(True, which="both", ls=":")
# Tô vùng khả kiến từ 0.38 đến 0.76 µm
plt.axvspan(0.38, 0.76, color="gray", alpha=0.3)

plt.show()

In [ ]:
u_lambda_integrate = integrate.trapezoid(data_ext["u_lambda"], data_ext["lambda"])
Td_1=16.4*(radius_cm[1]*1e5)**(-1./15)*(u_lambda_integrate/(8.6e-13))**(1/6)
print(u_lambda_integrate ,Td_1)

## Energy balance to solve Td

In [ ]:
def P_mean_opacity(Td, wavelengths, qabs_list):
    B_vals = planck(wavelengths, Td)
    numerator = integrate.trapezoid( qabs_list * B_vals, wavelengths)
    denominator = integrate.trapezoid(B_vals, wavelengths)
    return numerator / denominator

def solve_Td(df_combined, u_lambda, radius, Td_init, tol, max_iter,n,k):
    
    Q_abs, Q_ext, Q_sca = mie_theory(df = df_combined, 
                                     wavelength = "lambda", 
                                     n = n , 
                                     k = k , 
                                     radius = radius)
    wavelengths = df_combined["lambda"]
    
    # Tìm Td
    Td = Td_init
    
    for _ in range(max_iter):
        opacity_mP = P_mean_opacity(Td, wavelengths, Q_abs)

        heat = integrate.trapezoid(Q_abs * u_lambda, wavelengths) # erg/cm3
        cool = 4 * opacity_mP * o_SB   #  erg/cm²/s/K⁴

        T_new = (c * heat / cool)**0.25
        if abs(T_new - Td) < tol:
            return T_new
        Td = T_new

    print("Cảnh báo: không hội tụ trong", max_iter, "lần lặp.")
    return Td


In [ ]:
# For silicate core
from joblib import Parallel, delayed
import multiprocessing

num_cores = multiprocessing.cpu_count()

T_lambda_sili = Parallel(n_jobs=num_cores)(
    delayed(solve_Td)(
        df_interp,
        u_lambda,
        radius_cm[i],
        Td_init=10.0, tol=1e-9,
        max_iter=100,
        n="n_pyrmg",
        k="k_pyrmg"
    ) for i in range(len(radius_cm))
)

T_lambda_sili = np.array(T_lambda_sili, dtype=float)
T_lambda_sili = pd.Series(T_lambda_sili).dropna().to_numpy()


In [ ]:
# For mix with mf_m =0.2
from joblib import Parallel, delayed
import multiprocessing

num_cores = multiprocessing.cpu_count()

# U = 1
T_lambda = Parallel(n_jobs=num_cores)(
    delayed(solve_Td)(
        df_combined,
        u_lambda,
        radius_mix_cm[i],
        Td_init=10.0, tol=1e-9,
        max_iter=100,
        n="n",
        k="k"
    ) for i in range(len(radius_mix_cm))
)
T_lambda = np.array(T_lambda, dtype=float)
T_lambda = pd.Series(T_lambda).dropna().to_numpy()

# U = 10
T_lambda_10 = Parallel(n_jobs=num_cores)(
    delayed(solve_Td)(
        df_combined,
        u_lambda_10,
        radius_mix_cm[i],
        Td_init=10.0, tol=1e-9,
        max_iter=100,
        n="n",
        k="k"
    ) for i in range(len(radius_mix_cm))
)
T_lambda_10 = np.array(T_lambda_10, dtype=float)
T_lambda_10 = pd.Series(T_lambda_10).dropna().to_numpy()

# U = 20
T_lambda_20 = Parallel(n_jobs=num_cores)(
    delayed(solve_Td)(
        df_combined,
        u_lambda_20,
        radius_mix_cm[i],
        Td_init=10.0, tol=1e-9,
        max_iter=100,
        n="n",
        k="k"
    ) for i in range(len(radius_mix_cm))
)
T_lambda_20 = np.array(T_lambda_20, dtype=float)
T_lambda_20 = pd.Series(T_lambda_20).dropna().to_numpy()

# U = 50
T_lambda_50 = Parallel(n_jobs=num_cores)(
    delayed(solve_Td)(
        df_combined,
        u_lambda_50,
        radius_mix_cm[i],
        Td_init=10.0, tol=1e-9,
        max_iter=100,
        n="n",
        k="k"
    ) for i in range(len(radius_mix_cm))
)
T_lambda_50 = np.array(T_lambda_50, dtype=float)
T_lambda_50 = pd.Series(T_lambda_50).dropna().to_numpy()

# U = 100
T_lambda_100 = Parallel(n_jobs=num_cores)(
    delayed(solve_Td)(
        df_combined,
        u_lambda_100,
        radius_mix_cm[i],
        Td_init=10.0, tol=1e-9,
        max_iter=100,
        n="n",
        k="k"
    ) for i in range(len(radius_mix_cm))
)
T_lambda_100 = np.array(T_lambda_100, dtype=float)
T_lambda_100 = pd.Series(T_lambda_100).dropna().to_numpy()

# U = 1000
T_lambda_1000 = Parallel(n_jobs=num_cores)(
    delayed(solve_Td)(
        df_combined,
        u_lambda_1000,
        radius_mix_cm[i],
        Td_init=10.0, tol=1e-9,
        max_iter=100,
        n="n",
        k="k"
    ) for i in range(len(radius_mix_cm))
)
T_lambda_1000 = np.array(T_lambda_1000, dtype=float)
T_lambda_1000 = pd.Series(T_lambda_1000).dropna().to_numpy()

In [ ]:
# For mix with mf_m =0.5
from joblib import Parallel, delayed
import multiprocessing

num_cores = multiprocessing.cpu_count()

T_lambda_2 = Parallel(n_jobs=num_cores)(
    delayed(solve_Td)(
        df_combined_2,
        u_lambda,
        radius_mix_cm_2[i],
        Td_init=10.0, tol=1e-9,
        max_iter=100,
        n="n",
        k="k"
    ) for i in range(len(radius_mix_cm_2))
)
T_lambda_2 = np.array(T_lambda_2, dtype=float)
T_lambda_2 = pd.Series(T_lambda_2).dropna().to_numpy()

In [ ]:
# For mix with mf_m =1
from joblib import Parallel, delayed
import multiprocessing

num_cores = multiprocessing.cpu_count()

T_lambda_3 = Parallel(n_jobs=num_cores)(
    delayed(solve_Td)(
        df_combined_3,
        u_lambda,
        radius_mix_cm_3[i],
        Td_init=10.0, tol=1e-9,
        max_iter=100,
        n="n",
        k="k"
    ) for i in range(len(radius_mix_cm_3))
)
T_lambda_3 = np.array(T_lambda_3, dtype=float)
T_lambda_3 = pd.Series(T_lambda_3).dropna().to_numpy()

In [ ]:
# For mix with mf_m =1
from joblib import Parallel, delayed
import multiprocessing

num_cores = multiprocessing.cpu_count()

T_lambda_4 = Parallel(n_jobs=num_cores)(
    delayed(solve_Td)(
        df_combined_4,
        u_lambda,
        radius_mix_cm_4[i],
        Td_init=10.0, tol=1e-9,
        max_iter=100,
        n="n",
        k="k"
    ) for i in range(len(radius_mix_cm_4))
)
T_lambda_4 = np.array(T_lambda_4, dtype=float)
T_lambda_4 = pd.Series(T_lambda_4).dropna().to_numpy()

In [ ]:
# For mix with mf_m =1
from joblib import Parallel, delayed
import multiprocessing

num_cores = multiprocessing.cpu_count()

T_lambda_5 = Parallel(n_jobs=num_cores)(
    delayed(solve_Td)(
        df_combined_5,
        u_lambda,
        radius_mix_cm_5[i],
        Td_init=10.0, tol=1e-9,
        max_iter=100,
        n="n",
        k="k"
    ) for i in range(len(radius_mix_cm_5))
)
T_lambda_5 = np.array(T_lambda_5, dtype=float)
T_lambda_5 = pd.Series(T_lambda_5).dropna().to_numpy()

In [ ]:
# For mix with mf_m =1
from joblib import Parallel, delayed
import multiprocessing

num_cores = multiprocessing.cpu_count()

T_lambda_6 = Parallel(n_jobs=num_cores)(
    delayed(solve_Td)(
        df_combined_6,
        u_lambda,
        radius_mix_cm_6[i],
        Td_init=10.0, tol=1e-9,
        max_iter=100,
        n="n",
        k="k"
    ) for i in range(len(radius_mix_cm_6))
)
T_lambda_6 = np.array(T_lambda_6, dtype=float)
T_lambda_6 = pd.Series(T_lambda_6).dropna().to_numpy()

In [ ]:
# For mix with mf_m =1
from joblib import Parallel, delayed
import multiprocessing

num_cores = multiprocessing.cpu_count()

T_lambda_7 = Parallel(n_jobs=num_cores)(
    delayed(solve_Td)(
        df_combined_7,
        u_lambda,
        radius_mix_cm_7[i],
        Td_init=10.0, tol=1e-9,
        max_iter=100,
        n="n",
        k="k"
    ) for i in range(len(radius_mix_cm_7))
)
T_lambda_7 = np.array(T_lambda_7, dtype=float)
T_lambda_7 = pd.Series(T_lambda_7).dropna().to_numpy()

In [ ]:
# For mix with mf_m =1
from joblib import Parallel, delayed
import multiprocessing

num_cores = multiprocessing.cpu_count()

T_lambda_8 = Parallel(n_jobs=num_cores)(
    delayed(solve_Td)(
        df_combined_8,
        u_lambda,
        radius_mix_cm_8[i],
        Td_init=10.0, tol=1e-9,
        max_iter=100,
        n="n",
        k="k"
    ) for i in range(len(radius_mix_cm_8))
)
T_lambda_8 = np.array(T_lambda_8, dtype=float)
T_lambda_8 = pd.Series(T_lambda_8).dropna().to_numpy()

In [ ]:
# For mix with mf_m =1
from joblib import Parallel, delayed
import multiprocessing

num_cores = multiprocessing.cpu_count()

T_lambda_9 = Parallel(n_jobs=num_cores)(
    delayed(solve_Td)(
        df_combined_9,
        u_lambda,
        radius_mix_cm_9[i],
        Td_init=10.0, tol=1e-9,
        max_iter=100,
        n="n",
        k="k"
    ) for i in range(len(radius_mix_cm_9))
)
T_lambda_9 = np.array(T_lambda_9, dtype=float)
T_lambda_9 = pd.Series(T_lambda_9).dropna().to_numpy()

In [ ]:
# For mix with mf_m =1
from joblib import Parallel, delayed
import multiprocessing

num_cores = multiprocessing.cpu_count()

T_lambda_10 = Parallel(n_jobs=num_cores)(
    delayed(solve_Td)(
        df_combined_10,
        u_lambda,
        radius_mix_cm_10[i],
        Td_init=10.0, tol=1e-9,
        max_iter=100,
        n="n",
        k="k"
    ) for i in range(len(radius_mix_cm_10))
)
T_lambda_10 = np.array(T_lambda_10, dtype=float)
T_lambda_10 = pd.Series(T_lambda_10).dropna().to_numpy()

In [ ]:
# For mix with mf_m =1
from joblib import Parallel, delayed
import multiprocessing

num_cores = multiprocessing.cpu_count()

T_lambda_11 = Parallel(n_jobs=num_cores)(
    delayed(solve_Td)(
        df_combined_11,
        u_lambda,
        radius_mix_cm_11[i],
        Td_init=10.0, tol=1e-9,
        max_iter=100,
        n="n",
        k="k"
    ) for i in range(len(radius_mix_cm_11))
)
T_lambda_11 = np.array(T_lambda_11, dtype=float)
T_lambda_11 = pd.Series(T_lambda_11).dropna().to_numpy()

In [ ]:
# For mix with mf_m =1
from joblib import Parallel, delayed
import multiprocessing

num_cores = multiprocessing.cpu_count()

T_lambda_12 = Parallel(n_jobs=num_cores)(
    delayed(solve_Td)(
        df_combined_12,
        u_lambda,
        radius_mix_cm_12[i],
        Td_init=10.0, tol=1e-9,
        max_iter=100,
        n="n",
        k="k"
    ) for i in range(len(radius_mix_cm_12))
)
T_lambda_12 = np.array(T_lambda_12, dtype=float)
T_lambda_12 = pd.Series(T_lambda_12).dropna().to_numpy()

In [ ]:
# For mix with mf_m =1
from joblib import Parallel, delayed
import multiprocessing

num_cores = multiprocessing.cpu_count()

T_lambda_13 = Parallel(n_jobs=num_cores)(
    delayed(solve_Td)(
        df_combined_13,
        u_lambda,
        radius_mix_cm_13[i],
        Td_init=10.0, tol=1e-9,
        max_iter=100,
        n="n",
        k="k"
    ) for i in range(len(radius_mix_cm_13))
)
T_lambda_13 = np.array(T_lambda_13, dtype=float)
T_lambda_13 = pd.Series(T_lambda_13).dropna().to_numpy()

In [ ]:
# For mix with mf_m =1
from joblib import Parallel, delayed
import multiprocessing

num_cores = multiprocessing.cpu_count()

T_lambda_14 = Parallel(n_jobs=num_cores)(
    delayed(solve_Td)(
        df_combined_14,
        u_lambda,
        radius_mix_cm_14[i],
        Td_init=10.0, tol=1e-9,
        max_iter=100,
        n="n",
        k="k"
    ) for i in range(len(radius_mix_cm_14))
)
T_lambda_14 = np.array(T_lambda_14, dtype=float)
T_lambda_14 = pd.Series(T_lambda_14).dropna().to_numpy()

In [ ]:
plt.figure()
#plt.plot(radius_cm*1e4, T_lambda_sili, label = "Silicate core", color = "red", ls="--")
plt.plot(radius_mix_cm*1e4, T_lambda, label = f"Mixing, f_m = {mf_m[0]}, U = {U[0]}")
plt.plot(radius_mix_cm*1e4, T_lambda_10, label = f"Mixing, f_m = {mf_m[0]}, U ={U[1]}")
plt.plot(radius_mix_cm*1e4, T_lambda_20, label = f"Mixing, f_m = {mf_m[0]}, U ={U[2]}")
plt.plot(radius_mix_cm*1e4, T_lambda_50, label = f"Mixing, f_m = {mf_m[0]}, U ={U[3]}")
plt.plot(radius_mix_cm*1e4, T_lambda_100, label = f"Mixing, f_m = {mf_m[0]}, U ={U[4]}")
plt.plot(radius_mix_cm*1e4, T_lambda_1000, label = f"Mixing, f_m = {mf_m[0]}, U ={U[5]}")
#plt.plot(radius_mix_cm_2*1e4, T_lambda_2, label = f"Mixing, f_m = {mf_m[1]}")
#plt.plot(radius_mix_cm_3*1e4, T_lambda_3, label = f"Mixing, f_m = {mf_m[2]}")
#plt.plot(radius_mix_cm_4*1e4, T_lambda_4, label = f"Mixing, f_m = {mf_m[3]}")
#plt.plot(radius_mix_cm_5*1e4, T_lambda_5, label = f"Mixing, f_m = {mf_m[4]}")
#plt.plot(radius_mix_cm_6*1e4, T_lambda_6, label = f"Mixing, f_m = {mf_m[5]}")
#plt.plot(radius_mix_cm_7*1e4, T_lambda_7, label = f"Mixing, f_m = {mf_m[6]}")
plt.plot(radius_mix_cm_8*1e4, T_lambda_8, label = f"Mixing, f_m = {mf_m[7]}")
#plt.plot(radius_mix_cm_9*1e4, T_lambda_9, label = f"Mixing, f_m = {mf_m[8]}")
#plt.plot(radius_mix_cm_10*1e4, T_lambda_10, label = f"Mixing, f_m = {mf_m[9]}")
#plt.plot(radius_mix_cm_11*1e4, T_lambda_11, label = f"Mixing, f_m = {mf_m[10]}")
#plt.plot(radius_mix_cm_12*1e4, T_lambda_12, label = f"Mixing, f_m = {mf_m[11]}")
#plt.plot(radius_mix_cm_13*1e4, T_lambda_13, label = f"Mixing, f_m = {mf_m[12]}")
#plt.plot(radius_mix_cm_14*1e4, T_lambda_14, label = f"Mixing, f_m = {mf_m[13]}")
plt.plot(radius_cm*1e4, T_lambda_sili, label = "Silicate core", color = "black", ls="--")
plt.xscale("log")
plt.yscale("linear")
plt.xlabel(r"Radius [$\mu$m]")
plt.ylabel(r"T[K]")
#plt.xlim(1e-1,500)
#plt.ylim(1e-3,1e3)
plt.title(r"Tempeture of dust") #and mix $\mathrm{MgFeSiO_4} [3.74 g/cm^3]$
plt.legend()
plt.grid(True, which="both", ls=":")
plt.show()
# Bán kính càng cao nhiệt độ càng thấp???

In [ ]:
def tem_paper(Av, x):
    td = [11+5.7*np.tanh(0.61-np.log10(Av))]*x**(1/5.9)
    return td

In [ ]:
x_uv = np.logspace(np.log10(1e-1),np.log10(1e5), 200)*1e4 #erg cm^-2 s^-1
Av = 0

tempaper = tem_paper(Av,x_uv)


In [ ]:
tempaper

In [ ]:
plt.figure()
plt.plot(x_uv, tempaper)
plt.xscale("log")
plt.yscale("linear")
plt.xlabel(r" Interstellar radiation field flux [$erg$ $cm^{-2}$ $s^{-1}$]")
plt.ylabel(r"T[K]")
#plt.xlim(1e-1,500)
#plt.ylim(1e-3,1e3)
plt.title(r"Tempeture of dust") #and mix $\mathrm{MgFeSiO_4} [3.74 g/cm^3]$
plt.legend()
plt.grid(True, which="both", ls=":")
plt.show()

## Intensity of thermal dust absorption

In [ ]:
def intensity(df_combined, radius_list, temp_list, n, k):
    """
    Tính intensity I_lambda với phân bố kích thước hạt: dn/da ∝ a^{-3.5}

    Tham số:
        df_combined: DataFrame chứa 'lambda', 'n', 'k'
        radius_list: mảng bán kính hạt [cm]
        temp_list: danh sách nhiệt độ tương ứng [K]
        mie: đối tượng/hàm có .efficiencies_mx(m, x)
        planck: hàm Planck(λ, T)
        n, k: tên cột chiết suất
        c: hệ số phân bố (thường là tốc độ ánh sáng cgs)

    Trả về:
        intensity: mảng I_lambda theo từng bước sóng
    """
    intensity = []

    for i in range(len(df_combined['lambda'])):
        lam = df_combined['lambda'][i]
        m = df_combined[n][i] + 1j * df_combined[k][i]

        Q_each_r = []

        for r, Td in zip(radius_list, temp_list):
            x = 2 * np.pi * r / lam
            qext, qsca, qback, g = mie.efficiencies_mx(m, x)
            qabs = qext - qsca
            B = planck(lam, Td)

            # a^{-1.5} đã bao gồm từ phân bố và tiết diện
            Q_each_r.append(qabs * B * r**(-1.5))

        Q_each_r = np.array(Q_each_r)

        # Tích phân trên toàn bộ bán kính
        if len(Q_each_r) == len(radius_list):
            I_lam = c * np.trapz(Q_each_r, radius_list)
        else:
            print(f"[!] Lỗi chiều tại lambda = {lam}")
            I_lam = np.nan

        intensity.append(I_lam)
 
    intensity = np.array(intensity, dtype=float)

    return intensity


In [ ]:
from joblib import Parallel, delayed
import multiprocessing

num_cores = multiprocessing.cpu_count()

params = [
    (df_combined, radius_mix_cm, T_lambda, 'n', 'k'),
    (df_combined_2, radius_mix_cm_2, T_lambda_2, 'n', 'k'),
    (df_combined_3, radius_mix_cm_3, T_lambda_3, 'n', 'k'),
    (df_combined_4, radius_mix_cm_4, T_lambda_4, 'n', 'k'),
    (df_combined_5, radius_mix_cm_5, T_lambda_5, 'n', 'k'),
    (df_combined_6, radius_mix_cm_6, T_lambda_6, 'n', 'k'),
    (df_combined_7, radius_mix_cm_7, T_lambda_7, 'n', 'k'),
    (df_combined_8, radius_mix_cm_8, T_lambda_8, 'n', 'k'),
    (df_combined_9, radius_mix_cm_9, T_lambda_9, 'n', 'k'),
    (df_combined_10, radius_mix_cm_10, T_lambda_10, 'n', 'k'),
    (df_combined_11, radius_mix_cm_11, T_lambda_11, 'n', 'k'),
    (df_combined_12, radius_mix_cm_12, T_lambda_12, 'n', 'k'),
    (df_combined_13, radius_mix_cm_13, T_lambda_13, 'n', 'k'),
    (df_combined_14, radius_mix_cm_14, T_lambda_14, 'n', 'k'),]

results = Parallel(n_jobs=num_cores)(
    delayed(intensity)(df, r, T, n, k)
    for df, r, T, n, k in params
)

intensity_lambda, \
intensity_lambda_2, \
intensity_lambda_3, \
intensity_lambda_4, \
intensity_lambda_5, \
intensity_lambda_6, \
intensity_lambda_7, \
intensity_lambda_8, \
intensity_lambda_9, \
intensity_lambda_10, \
intensity_lambda_11, \
intensity_lambda_12, \
intensity_lambda_13, \
intensity_lambda_14= results

In [ ]:
from joblib import Parallel, delayed
import multiprocessing

def intensity_parallel(df_combined, radius_list, temp_list, n, k):
    def calc_intensity_at_lambda(i):
        lam = df_combined['lambda'][i]
        m = df_combined[n][i] + 1j * df_combined[k][i]
        Q_each_r = []
        for r, Td in zip(radius_list, temp_list):
            x = 2 * np.pi * r / lam
            qext, qsca, qback, g = mie.efficiencies_mx(m, x)
            qabs = qext - qsca
            B = planck(lam, Td)
            Q_each_r.append(qabs * B * r**(-1.5))
        Q_each_r = np.array(Q_each_r)
        if len(Q_each_r) == len(radius_list):
            I_lam = c * np.trapz(Q_each_r, radius_list)
        else:
            I_lam = np.nan
        return I_lam

    num_cores = multiprocessing.cpu_count()
    intensity = Parallel(n_jobs=num_cores)(
        delayed(calc_intensity_at_lambda)(i) for i in range(len(df_combined['lambda']))
    )
    return np.array(intensity, dtype=float)

# Sử dụng:
intensity_lambda_sili = intensity_parallel(
    df_interp,
    radius_cm,
    T_lambda_sili,
    'n_pyrmg',
    'k_pyrmg'
)

In [ ]:
# Vẽ đồ thị
plt.figure(figsize=(14, 6))
plt.plot(df_combined["lambda"]*1e4, intensity_lambda, label = f"Mixing, f_m = {mf_m[0]}") # label =f"Mixing with mf_m = {mf_m:.2f}
plt.plot(df_combined_2["lambda"]*1e4, intensity_lambda_2, label = f"Mixing, f_m = {mf_m[1]}") 
plt.plot(df_combined_3["lambda"]*1e4, intensity_lambda_3, label = f"Mixing, f_m = {mf_m[2]}") 
plt.plot(df_combined_4["lambda"]*1e4, intensity_lambda_4, label = f"Mixing, f_m = {mf_m[3]}") 
plt.plot(df_combined_5["lambda"]*1e4, intensity_lambda_5, label = f"Mixing, f_m = {mf_m[4]}")
plt.plot(df_combined_6["lambda"]*1e4, intensity_lambda_6, label = f"Mixing, f_m = {mf_m[5]}")  
plt.plot(df_combined_7["lambda"]*1e4, intensity_lambda_7, label = f"Mixing, f_m = {mf_m[6]}")
plt.plot(df_combined_8["lambda"]*1e4, intensity_lambda_8, label = f"Mixing, f_m = {mf_m[7]}")
plt.plot(df_combined_9["lambda"]*1e4, intensity_lambda_9, label = f"Mixing, f_m = {mf_m[8]}")
plt.plot(df_combined_10["lambda"]*1e4, intensity_lambda_10, label = f"Mixing, f_m = {mf_m[9]}")
plt.plot(df_combined_11["lambda"]*1e4, intensity_lambda_11, label = f"Mixing, f_m = {mf_m[10]}")
plt.plot(df_combined_12["lambda"]*1e4, intensity_lambda_12, label = f"Mixing, f_m = {mf_m[11]}")
plt.plot(df_combined_13["lambda"]*1e4, intensity_lambda_13, label = f"Mixing, f_m = {mf_m[12]}")
plt.plot(df_combined_14["lambda"]*1e4, intensity_lambda_14, label = f"Mixing, f_m = {mf_m[13]}")
# Vẽ đường cho silicate core
plt.plot(df_interp["lambda"]*1e4, intensity_lambda_sili , label = "Silicat core", color = "black", ls = "--") 
plt.xscale("log")
plt.yscale("log")
plt.xlabel(r"$\lambda$ [$\mu$m]")
plt.ylabel(r"$\lambda I_{\mathrm{\lambda}}$ / $N_H$ [erg $s^{-1}$ $sr^{-1}$ $H^{-1}$]")
plt.xlim(13,1e2)
plt.ylim(1e4,1e13)
plt.title(r"Intensity of mixing and silicate core r ") #and mix $\mathrm{MgFeSiO_4} [3.74 g/cm^3]$
plt.legend()
plt.grid(True, which="both", ls=":")
# Tô vùng khả kiến từ 0.38 đến 0.76 µm
#plt.axvspan(0.38, 0.76, color="gray", alpha=0.3)

plt.show()